In [55]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()

API_KEY = os.getenv("API_KEY")

# URLs to get diferent calls from API services
url_trip = f"https://api.resrobot.se/v2.1/trip?format=json&originId=740000002&destId=740000003&passlist=true&showPassingPoints=true&accessId={API_KEY}"

url_stopId = f"https://api.resrobot.se/v2.1/departureBoard?id=740000002&format=json&accessId={API_KEY}"

url_stopName = f"https://api.resrobot.se/v2.1/location.name?input=Göteborg&format=json&accessId={API_KEY}"

url_arrivals = f"https://api.resrobot.se/v2.1/arrivalBoard?id=740000002&format=json&accessId={API_KEY}"

response = requests.get(url_stopName) # Chooses which url is shown
result = response.json()
result.keys()

dict_keys(['stopLocationOrCoordLocation', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'requestId'])

In [ ]:
# Shows the first key when url is 'url_trip'
result["Trip"][0]["LegList"]

In [58]:
# Gives us the stoplocations name when url above is 'url_stopName'
result["stopLocationOrCoordLocation"][0]["StopLocation"]["name"]

'GÖTEBORG'

In [59]:
# Function to extract the stopname
def get_stop_name(location):
    url_stopName = f"https://api.resrobot.se/v2.1/location.name?input={location}&format=json&accessId={API_KEY}"

    result_name = requests.get(url_stopName)
    return result_name.json().get("stopLocationOrCoordLocation")

stop_name = get_stop_name("Trätorget")
print(stop_name)

[{'StopLocation': {'productAtStop': [{'icon': {'res': 'prod_gen'}, 'cls': '128'}], 'timezoneOffset': 60, 'id': 'A=1@O=Trätorget (Göteborg kn)@X=12053177@Y=57715439@U=1@L=740059660@B=1@p=1737347805@', 'extId': '740059660', 'name': 'Trätorget (Göteborg kn)', 'lon': 12.053177, 'lat': 57.715439, 'weight': 533, 'products': 128, 'minimumChangeDuration': 'PT0S'}}, {'CoordLocation': {'links': [{'link': [{'rel': 'refine', 'href': 'https://api.resrobot.se/v2.1/location.name?input=Tr%C3%A4torget%2C+Falk%C3%B6ping&refineId=A%3D2%40O%3DTr%C3%A4torget%2C+Falk%C3%B6ping%40X%3D13554412%40Y%3D58160352%40U%3D174%40b%3D980141995%40B%3D1%40p%3D1479298166%40&type=A'}]}], 'icon': {'res': 'loc_addr'}, 'id': 'A=2@O=Trätorget, Falköping@X=13554412@Y=58160352@U=174@b=980141995@B=1@p=1479298166@', 'name': 'Trätorget, Falköping', 'type': 'ADR', 'lon': 13.554412, 'lat': 58.160352, 'refinable': True}}, {'CoordLocation': {'links': [{'link': [{'rel': 'refine', 'href': 'https://api.resrobot.se/v2.1/location.name?input

In [31]:
stop_name[0].get("StopLocation")

{'productAtStop': [{'icon': {'res': 'prod_gen'}, 'cls': '128'}],
 'timezoneOffset': 60,
 'id': 'A=1@O=Trätorget (Göteborg kn)@X=12053177@Y=57715439@U=1@L=740059660@B=1@p=1737347805@',
 'extId': '740059660',
 'name': 'Trätorget (Göteborg kn)',
 'lon': 12.053177,
 'lat': 57.715439,
 'weight': 533,
 'products': 128,
 'minimumChangeDuration': 'PT0S'}

In [32]:
# Gets stoplocation and if none, gets empty dict
station_id = stop_name[0].get("StopLocation", {}).get("extId")
print(station_id)

740059660


In [136]:
def get_departures(location_ids, max_results=8):
    url_departures = f"https://api.resrobot.se/v2.1/departureBoard?id={location_ids}&format=json&maxJourneys={max_results}&accessId={API_KEY}"

    response = requests.get(url_departures)
    data = response.json()
    departures = data.get("Departure", [])

    return departures[:max_results]

# get_departures(740059660)

In [137]:
import re

def get_timetable_dep(station):
    if station:
        station_name = station[0]["StopLocation"]["name"]
        station_id_raw = station[0]["StopLocation"]["id"]

        match = re.search(r"L=(\d+)", station_id_raw) # Looks for the key L that holds the station ID
        if match:
            station_id = match.group(1)
        else:
            print("Kunde inte extrahera stationens ID.")
            station_id = None
        print(f"Stationens namn: {station_name}")
        print(f"Stations id: {station_id}")

        departures = get_departures(station_id, max_results=8) # Only gives the first 8 results, also here we decide number of results we want

        if departures:
            for departure in departures:
                print(f"Tid: {departure['time']}, Destination: {departure['direction']}")
        else:
            print("Inga avgångar")
    else:
        print("Stationen hittades inte")

get_timetable_dep(station=get_stop_name("Göteborg C"))

Stationens namn: Göteborg Centralstation
Stations id: 740000002
Tid: 10:56:00, Destination: Höga hallar (Härryda kn)
Tid: 10:56:00, Destination: Stenungsund station
Tid: 10:56:00, Destination: Göteborg Östra sjukhuset
Tid: 10:56:00, Destination: Särö centrum (Kungsbacka kn)
Tid: 10:56:00, Destination: Kortedala Aprilgatan (Göteborg kn)
Tid: 10:59:00, Destination: Kungälv resecentrum
Tid: 10:59:00, Destination: Fyrktorget (Göteborg kn)
Tid: 10:59:00, Destination: Stockholm Centralstation


In [46]:
station = get_stop_name(input("Hållplats: "))
get_timetable_dep()

Stationens namn: Trätorget (Göteborg kn)
Stations id: 740059660
Tid: 21:14:00, Destination: Hinnebäcksgatan (Göteborg kn)
Tid: 21:23:00, Destination: Göteborg Östra sjukhuset
Tid: 21:29:00, Destination: Hinnebäcksgatan (Göteborg kn)
Tid: 21:38:00, Destination: Göteborg Östra sjukhuset
Tid: 21:44:00, Destination: Hinnebäcksgatan (Göteborg kn)
Tid: 21:53:00, Destination: Göteborg Östra sjukhuset
Tid: 21:59:00, Destination: Hinnebäcksgatan (Göteborg kn)
Tid: 22:08:00, Destination: Göteborg Östra sjukhuset


In [138]:
def get_arrivials(location_ids, max_results=8):
    url_arrivals = f"https://api.resrobot.se/v2.1/arrivalBoard?id={location_ids}&format=json&maxJourneys={max_results}&accessId={API_KEY}"
    response = requests.get(url_arrivals)
    
    data_arrivals = response.json()
    arrivals = data_arrivals.get("Arrival", [])
    return arrivals[:max_results] # Gets the first 8 results

In [ ]:

def get_timetable_arr(station):
    if station:
        station_name = station[0]["StopLocation"]["name"]
        station_id_raw = station[0]["StopLocation"]["id"]

        match = re.search(r"L=(\d+)", station_id_raw)
        if match:
            station_id = match.group(1)
        else:
            print("Kunde inte extrahera stationens ID.")
            station_id = None
        print(f"Stationens namn: {station_name}")
        print(f"Stations id: {station_id}")

        arrivals = get_arrivials(station_id)
        if arrivals:
            print(f"Ankomster till {station_name}:")
            for arrival in arrivals:
                time = arrival.get('time', 'N/A')
                origin = arrival.get('origin', 'N/A')
                transport = arrival.get('ProductAtStop', {}).get('displayNumber', 'Okänt fordon')
                print(f"Tid: {time}, Avgång: {origin}, FordonsNr: {transport}")
        else:
            print("Inga avgångar")
    else:
        print("Stationen hittades inte")

get_timetable_arr(station = get_stop_name("Göteborg C"))

Stationens namn: Göteborg Centralstation
Stations id: 740000002
Ankomster till Göteborg Centralstation:
Tid: 22:11:00, Avgång: Göteborg Eketrägatan, FordonsNr: 21
Tid: 22:13:00, Avgång: Mölnlycke station (Härryda kn), FordonsNr: X4
Tid: 22:13:00, Avgång: Partille centrum, FordonsNr: X1
Tid: 22:14:00, Avgång: Kungälv resecentrum, FordonsNr: X4
Tid: 22:14:00, Avgång: Mölnlycke station (Härryda kn), FordonsNr: X4
Tid: 22:15:00, Avgång: Göteborg Heden, FordonsNr: 510
Tid: 22:15:00, Avgång: Kungsbacka station, FordonsNr: 3086
Tid: 22:16:00, Avgång: Kungälv resecentrum, FordonsNr: X4


In [128]:

get_timetable_dep(station = get_stop_name("Göteborg C"))

Stationens namn: Göteborg Centralstation
Stations id: 740000002
Tid: 22:11:00, Destination: Merkuriusgatan (Göteborg kn)
Tid: 22:13:00, Destination: Kungälv resecentrum
Tid: 22:13:00, Destination: Hornkamsgatan (Göteborg kn)
Tid: 22:14:00, Destination: Höga hallar (Härryda kn)
Tid: 22:14:00, Destination: Kungälv resecentrum
Tid: 22:15:00, Destination: Partille centrum
Tid: 22:15:00, Destination: Kungsbacka station
Tid: 22:16:00, Destination: Höga hallar (Härryda kn)


In [139]:
import re

class get_stops():

    station = get_stop_name("Trätorget")

    def get_departures(location_ids, max_results=8):
        url_departures = f"https://api.resrobot.se/v2.1/departureBoard?id={location_ids}&format=json&maxJourneys={max_results}&accessId={API_KEY}"

        response = requests.get(url_departures)
        data = response.json()
        departures = data.get("Departure", [])

        return departures[:max_results]

    def get_timetable_dep(station):
        if station:
            station_name = station[0]["StopLocation"]["name"]
            station_id_raw = station[0]["StopLocation"]["id"]

            match = re.search(r"L=(\d+)", station_id_raw)
            if match:
                station_id = match.group(1)
            else:
                print("Kunde inte extrahera stationens ID.")
                station_id = None
            print(f"Stationens namn: {station_name}")
            print(f"Stations id: {station_id}")

            departures = get_departures(station_id, max_results=8)

            if departures:
                for departure in departures:
                    transport = departure.get('ProductAtStop', {}).get('displayNumber', 'Okänt fordon')
                    print(f"Tid: {departure['time']}, Destination: {departure['direction']}, Linje: {transport}")
            else:
                print("Inga avgångar")
        else:
            print("Stationen hittades inte")

    print("------------DEPARTURE--------------")

    get_timetable_dep(station=get_stop_name("Göteborg C"))

    print("------------ARRIVAL--------------")

    def get_arrivials(location_ids, max_results=8):
        url_arrivals = f"https://api.resrobot.se/v2.1/arrivalBoard?id={location_ids}&format=json&maxJourneys={max_results}&accessId={API_KEY}"
        response = requests.get(url_arrivals)
        
        data_arrivals = response.json()
        arrivals = data_arrivals.get("Arrival", [])
        return arrivals[:max_results]
    
    def get_timetable_arr(station):
        if station:
            station_name = station[0]["StopLocation"]["name"]
            station_id_raw = station[0]["StopLocation"]["id"]

            match = re.search(r"L=(\d+)", station_id_raw)
            if match:
                station_id = match.group(1)
            else:
                print("Kunde inte extrahera stationens ID.")
                station_id = None
            print(f"Stationens namn: {station_name}")
            print(f"Stations id: {station_id}")

            arrivals = get_arrivials(station_id, max_results=8)

            if arrivals:
                for arrival in arrivals:
                    transport = arrival.get('ProductAtStop', {}).get('displayNumber', 'Okänt fordon')
                    print(f"Tid: {arrival['time']}, Origin: {arrival['origin']}, Linje: {transport}")
            else:
                print("Inga avgångar")
        else:
            print("Stationen hittades inte")

    get_timetable_arr(station = get_stop_name("Göteborg C"))
    

------------DEPARTURE--------------
Stationens namn: Göteborg Centralstation
Stations id: 740000002
Tid: 10:59:00, Destination: Kungälv resecentrum, Linje: X4
Tid: 10:59:00, Destination: Fyrktorget (Göteborg kn), Linje: 16
Tid: 10:59:00, Destination: Stockholm Centralstation, Linje: 2032
Tid: 10:59:00, Destination: Partille centrum, Linje: X1
Tid: 10:59:00, Destination: Göteborg Varmfrontsgatan, Linje: 6
Tid: 11:00:00, Destination: Kungälv resecentrum, Linje: X4
Tid: 11:00:00, Destination: Bockkranen (Göteborg kn), Linje: 16
Tid: 11:00:00, Destination: Gamlestads Torg Västra (Göteborg kn), Linje: 59
------------ARRIVAL--------------
Stationens namn: Göteborg Centralstation
Stations id: 740000002
Tid: 10:59:00, Origin: Höga hallar (Härryda kn), Linje: X4
Tid: 10:59:00, Origin: Bockkranen (Göteborg kn), Linje: 16
Tid: 10:59:00, Origin: Hornkamsgatan (Göteborg kn), Linje: X1
Tid: 10:59:00, Origin: Kortedala Aprilgatan (Göteborg kn), Linje: 6
Tid: 11:00:00, Origin: Höga hallar (Härryda kn)